## Seq2Seq

> 챗봇, 번역, 이미지 캡셔닝등에 사용되는 시퀀스 학습/생성 모델인 Seq2Seq 을 구현해본다. 영어 단어를 한국어 단어로 번역하는 프로그램을 만들어본다.

In [19]:
import numpy as np
import tensorflow as tf

# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4 인 경우
#       word -> ['w', 'o', 'r', 'd']
#       to   -> ['t', 'o', 'P', 'P']

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 영어를 한글로 번역하기 위한 학습 데이터
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    
    for seq in seq_data:
        # 인코더 셀의 입력값, 입력단어의 글자들을 한글자씩 떼어 배열로 만듦
        input_ = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값, 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 학습을 위해 비교할 디코더 셀의 출력값, 끝나는 것을 알려주기 위해 마지막에 E를 붙인다.
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input_])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    return input_batch, output_batch, target_batch

In [20]:
tf.reset_default_graph()
###################
# Hyper-Parameter #
###################
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
n_class = n_input = dic_len

###################
# seq2seq 모델 구성 #
###################
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
# [batch_size, time_step, input_size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
# [batch_size, time_steps]
labels = tf.placeholder(tf.int64, [None, None])

# 인코더 셀을 구성
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)
    
# 디코더 셀을 구성
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    
    # Seq2Seq 모델은 인코더 셀의 최종 상태값을
    # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)
    
logits = tf.layers.dense(outputs, n_class, activation=None)

# loss & optimizer
cross_entropy = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [26]:
####################
# Seq2Seq Training #
####################
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(1, total_epoch+1):
    loss, _ = sess.run([cross_entropy, train_op],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  labels: target_batch})

    print('Epoch: {:04d}'.format(epoch), 
          'Loss: {:.5f}'.format(loss))
        
print('최적화 완료!')

Epoch: 0001 Loss: 3.71943
Epoch: 0002 Loss: 3.57409
Epoch: 0003 Loss: 3.35244
Epoch: 0004 Loss: 2.95590
Epoch: 0005 Loss: 2.27670
Epoch: 0006 Loss: 2.85396
Epoch: 0007 Loss: 1.92651
Epoch: 0008 Loss: 1.79554
Epoch: 0009 Loss: 1.84778
Epoch: 0010 Loss: 1.71881
Epoch: 0011 Loss: 1.58287
Epoch: 0012 Loss: 1.37200
Epoch: 0013 Loss: 1.09034
Epoch: 0014 Loss: 1.04822
Epoch: 0015 Loss: 1.04123
Epoch: 0016 Loss: 0.65715
Epoch: 0017 Loss: 0.71100
Epoch: 0018 Loss: 0.68041
Epoch: 0019 Loss: 0.50321
Epoch: 0020 Loss: 0.80881
Epoch: 0021 Loss: 0.56269
Epoch: 0022 Loss: 0.61115
Epoch: 0023 Loss: 0.35851
Epoch: 0024 Loss: 0.40565
Epoch: 0025 Loss: 0.26331
Epoch: 0026 Loss: 0.27602
Epoch: 0027 Loss: 0.20061
Epoch: 0028 Loss: 0.35559
Epoch: 0029 Loss: 0.31538
Epoch: 0030 Loss: 0.26647
Epoch: 0031 Loss: 0.21907
Epoch: 0032 Loss: 0.22060
Epoch: 0033 Loss: 0.18221
Epoch: 0034 Loss: 0.10547
Epoch: 0035 Loss: 0.16539
Epoch: 0036 Loss: 0.13805
Epoch: 0037 Loss: 0.11933
Epoch: 0038 Loss: 0.06234
Epoch: 0039 

In [27]:
################
# Seq2Seq Test #
################
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채운다.
    # ['word', 'PPPP']
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(logits, 2)
    
    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 labels: target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    decoded = [char_arr[i] for i in result[0]]

    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated


print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))    


=== 번역 테스트 ===
word -> 단어
wodr -> 단어
love -> 사랑
loev -> 사랑
abcd -> 놀이
